In [5]:
using DataFrames
using Random

# TrainTestSplit

In [6]:
function tts(df::DataFrame; at = .75)
    sample = randsubseq(1:size(df,1), at)
    trainingset = df[sample, :]
    notsample = [i for i in 1:size(df,1) if isempty(searchsorted(sample, i))]
    testset = df[notsample, :]
    return(trainingset,testset)
end

tts (generic function with 1 method)

In [7]:
df = DataFrame(:A => randn(20), :B => randn(20))

,A,B
,Float64,Float64
1,-0.00013721,-0.608496
2,2.19545,0.057122
3,-1.07612,0.807587
4,0.00116801,-2.57121
5,0.376163,-1.11479
6,-0.431085,-0.173849
7,-1.3453,-0.64697
8,0.825617,-1.06371
9,-0.0559926,0.574496


In [9]:
train, test = tts(df); show(train); show(test)

16×2 DataFrame
 Row │ A            B         
     │ Float64      Float64   
─────┼────────────────────────
   1 │ -0.00013721  -0.608496
   2 │  2.19545      0.057122
   3 │ -1.07612      0.807587
   4 │  0.00116801  -2.57121
   5 │ -0.431085    -0.173849
   6 │  0.825617    -1.06371
   7 │ -0.0559926    0.574496
   8 │  2.65238      1.34499
   9 │  0.598919     0.188288
  10 │ -1.76991      0.727884
  11 │ -0.817312     0.592076
  12 │  0.319003    -0.495463
  13 │ -0.125543    -1.53332
  14 │  1.5144       0.156553
  15 │  0.394596    -0.763952
  16 │ -1.26985      0.2246174×2 DataFrame
 Row │ A          B        
     │ Float64    Float64  
─────┼─────────────────────
   1 │  0.376163  -1.11479
   2 │ -1.3453    -0.64697
   3 │ -0.161914   1.32221
   4 │  1.69617   -1.42877

# Standard Scaler

In [13]:
using Lathe.stats: mean, std

┌ Info: Precompiling Lathe [38d8eb38-e7b1-11e9-0012-376b6c802672]
└ @ Base loading.jl:1423
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `Lathe.models.PowerLog :: Tuple{Float64, Float64}` in module `Lathe.models`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `Lathe.models.MeanBaseline :: Tuple{Any}` in module `Lathe.models`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `Lathe.models.ClassBaseline :: Tuple{Any}` in module `Lathe.models`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `Lathe.models.Pipeline :: Tuple{Any}` in module `Lathe.models`
└ @ Base.Docs docs/Docs.jl:240
  ** incremental compilation may be 

In [24]:
function standardscale(x::Vector{<:Real})
    N::Int64 = length(x)
    σ::Number = std(x)
    μ::Number = mean(x)
    Vector{Real}([i = (i-μ) / σ for i in x])
end

standardscale (generic function with 1 method)

In [25]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [26]:
standardscale(x)

10-element Vector{Real}:
 -1.9188064472004938
 -1.4924050144892729
 -1.066003581778052
 -0.6396021490668312
 -0.21320071635561041
  0.21320071635561041
  0.6396021490668312
  1.066003581778052
  1.4924050144892729
  1.9188064472004938

In [27]:
x = [75, 2, 3, 4, 5, 6, 7, 8, 9, 10]

10-element Vector{Int64}:
 75
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [28]:
standardscale(x)

10-element Vector{Real}:
 17.290069740400327
 -3.0348109528238902
 -2.756387929629038
 -2.4779649064341855
 -2.199541883239333
 -1.9211188600444808
 -1.6426958368496287
 -1.3642728136547764
 -1.085849790459924
 -0.8074267672650718

In [42]:
function onehot(df::DataFrame, symb::Symbol)
    dfcopy = copy(df)
    [dfcopy[!, Symbol(c)] = dfcopy[!,symb] .== c for c in unique(dfcopy[!,symb])]
    dfcopy
end

onehot (generic function with 2 methods)

In [43]:
df = DataFrame(:A => ["strawberry", "vanilla", "vanilla", "mango"], 
               :B => [1, 2, 3, 4])

,A,B
,String,Int64
1,strawberry,1
2,vanilla,2
3,vanilla,3
4,mango,4


In [44]:
encoded_df = onehot(df, :A)

,A,B,strawberry,vanilla,mango
,String,Int64,Bool,Bool,Bool
1,strawberry,1,1,0,0
2,vanilla,2,0,1,0
3,vanilla,3,0,1,0
4,mango,4,0,0,1


In [49]:
function ordinal(df::DataFrame, symb::Symbol)
    lookup = Dict(v => i for (i,v) in df[!, symb] |> unique |> enumerate)
    map(x->lookup[x], df[!, symb])
end

ordinal (generic function with 1 method)

In [50]:
ordinal(df, :A)

4-element Vector{Int64}:
 1
 2
 2
 3

In [101]:
function float_encode(df::DataFrame, symb::Symbol)
    floatencoded = Vector{Float64}()
    for observation in df[!, symb]
        s = ""
        [s = s * string(float(c)) for c in observation]
        s = replace(s, "." => "")
        push!(floatencoded, parse(Float64, s))
    end
    floatencoded
end

float_encode (generic function with 1 method)

In [102]:
float_encode(df, :A)

4-element Vector{Float64}:
 1.1501160114097013e37
 1.1809701100105011e25
 1.1809701100105011e25
 1.090970110010301e18

In [103]:
df

,A,B,A_encoded
,String,Int64,String
1,strawberry,1,
2,vanilla,2,
3,vanilla,3,
4,mango,4,


In [111]:
missing_mean(x::Vector) = begin
    summation = 0
    len = 0
    for number in x
        if ismissing(number)
            continue
        end
        summation += number
        len += 1
    end
    summation / len
end

missing_mean (generic function with 1 method)

In [115]:
function impute_mean(x::AbstractVector)
    μ::Number = missing_mean(x)
    [if ismissing(v) μ else v end for v in x]
end

impute_mean (generic function with 2 methods)

In [116]:
x = [5, 10, missing, 20, missing, 82, missing, missing, 40]

9-element Vector{Union{Missing, Int64}}:
  5
 10
   missing
 20
   missing
 82
   missing
   missing
 40

In [117]:
impute_mean(x)

9-element Vector{Real}:
  5
 10
 31.4
 20
 31.4
 82
 31.4
 31.4
 40